## Import investigator package of the PyOTIC software

In [ ]:
# Import os to easily join names of filepaths
import os

# Add the path of the PyOTIC Software to the system path
# Adjust this path to where the PyOTIC Software package is located
import sys
sys.path.append('../../')

#Load investigator package
import pyoti
pyoti.info()

#Create an experiment and open a new temporary experiment dbfile
experiment = pyoti.open_experiment()

## Create experiment file (or open previously saved one)

In [ ]:
# Choose the path the data should be loaded from
#
# datadir: The path where the data is located
# datafile: The name of the file that contains the data.
datadir = '../data/'
datafile = 'B01.bin'

In [ ]:
# Define a name for the record (defaults to 'default')
name='alpha'

# Choose the file, where standard values for the Record are defined
cfgfile = '../pyoti/etc/record/ASWAD.cfg'

experiment.create_record(name=name, cfgfile=cfgfile, filename=datafile, directory=datadir)

In [ ]:
name = 'used'
group = 'selection'
parent = 'alpha'

experiment.add_group(name, parent, group_type=group)

# Simple and fast step detection for SNR >> 1.0

In [ ]:
# Select region to get data from
region = 'used'

# Get region and motion object, to detect the steps and dwells
region = experiment.region(region)
motion = pyoti.create_motion(region=region, resolution=40000, impose_dwell_time=4.0, shift_time=0.1)

# If you want to include the very first and the very last plateau, uncomment
#motion.include_first_last_plateau=True

# Show the autodetected plateaus
motion.init_rfig(legend=True)

# Advanced and slower (still very fast) step detection for SNR <= 1.0 (or better)

In [ ]:
# import necessary modules and functions 
from pyoti.evaluate.step_finder import simulate_steps
from pyoti.evaluate.step_finder import filter_find_analyse_steps, plot_result
from matplotlib import pyplot as plt
# Set the standard figure size to 9 x 9 inch
from pylab import rcParams
rcParams['figure.figsize'] = 9, 9

In [ ]:
### Either, get data from the region
data = region.get_data('psdX')[:,0]
resolution = region.samplingrate

In [ ]:
### Or, simulate step data
duration = 1.0  # s
resolution = 40000.0  # Hz
dwell_time = 0.050  # s
SNR = 0.5
simulated_steps = simulate_steps(duration=duration, resolution=resolution,
                                 dwell_time=dwell_time, SNR=SNR)
data = simulated_steps.data + simulated_steps.noise

In [ ]:
### Filter the data, find the steps and plot the result
### Filter data
filter_time =  None  # None or s
filter_min_t = 0.001  # None or s
filter_max_t = 0.015  # None or s
filter_number = 25  # None or number
edginess = 3  # int
### Find steps in fbnl filtered data
expected_min_step_size = 8.0  # in values of data
expected_min_dwell_t = None  # None or s
step_size_threshold = None  # None (equal to 'adapt'), 'constant', or in values of data

step_finder_result, fig1 \
    = filter_find_analyse_steps(data, resolution, filter_time, filter_min_t, filter_max_t,
                                filter_number, edginess,
                                expected_min_step_size, expected_min_dwell_t,
                                step_size_threshold, cap_data=True,
                                verbose=True, plot=True)

### Plot data and step finder result
fig2, fig3 = plot_result(step_finder_result)#, simulated_steps)
plt.show(fig1)
plt.show(fig2)
plt.show(fig3)